# To Create a network for domestic and wild mammals and thier viruses

In [75]:
import sys
print(sys.version)

#Python 3.7.4 (default, Aug 9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
#pygbif 0.3.0

3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]


In [76]:
import pandas as pd
#import re
#import matplotlib.pyplot as plt
from pygbif import species
#from pygbif import occurrences as occ
import sqlite3
from sqlite3 import Error
#import pydot
#import math

### Setting up some parameters

In [77]:
# If you want to exclude an interaction add it here
#interactionsToExclude = "visitsFlowersOf" #If you're not intersted in pollinators this will remove most of them
interactionsToExclude = "interactsWith"    #Many interactions in GloBI do not specify a specific interaction. You can get a cleaner netwot by excluding them
#interactionsToExclude = ""

In [78]:
## A script to create globi.db is here https://github.com/AgentschapPlantentuinMeise/createGlobiDB
## Define the place to find the interaction data
globiDB = r"..\..\createGlobiDB\globi.db"

### Define the taxon for the notebook for which to find all interactions


In [79]:
# Species with some suseptability to SARS‐CoV‐2 from ...
# Gryseels, S., De Bruyn, L., Gyselings, R., Calvignac‐Spencer, S., Leendertz, F.H. and Leirs, H. (2020), Risk of human‐to‐wildlife transmission of SARS‐CoV‐2. Mam. Rev.. https://doi.org/10.1111/mam.12225

taxa={"Chiroptera": 0}

## Check to see if the taxon exits in GBIF

In [80]:
for animal in taxa:
    print("Animal = ", animal)
    try:
        #NOTE: The function method name_suggest has a different behavior with regards to the gender of Latin names to name_backbone
        # If name_backbone is given a name in one gender it can still return the best match from the GBIF backbone if that
        # name is spelled as if it was another gender.
        #key = species.name_suggest(q=taxon, limit = 1)
        match = species.name_backbone(name=animal, limit = 1)
        #print(key)

        # if there is no match this is returned form .name_backbone {'confidence': 100, 'matchType': 'NONE', 'synonym': False}
        if match['matchType'] == 'NONE':
            raise ValueError("TAXON NOT FOUND ON GBIF!")
        else:
            taxa[animal] = match['usageKey']
            print('The taxon to be studied is ' + match['scientificName'])
    except ValueError as ve:
        print(ve)
        exit(1)




Animal =  Chiroptera
The taxon to be studied is Chiroptera


### Just for information take a look at the interactions that the taxon has

In [81]:
# What are all the types of interactions involving taxon as source taxon?
#data[data['sourceTaxonName'] == taxon]['interactionTypeName'].unique()
try:
    connGlobi = sqlite3.connect(globiDB)
except Error as e:
    print(e)

In [84]:
curGlobi = connGlobi.cursor()
curGlobi.execute("SELECT interactionTypeName, COUNT(interactionTypeName) from globi  WHERE sourceTaxonKingdomName = 'Orthornavirae' and targetTaxonKingdomName = 'Mammalia' and sourceTaxonSpeciesName = 'Bos taurus' GROUP BY interactionTypeName;")
sourceInteractions = curGlobi.fetchall()

In [85]:
sourceInteractions

[]

## Get the primary interation data for the species in question

This is limited to the rank of species because otherwise the network can get very large with rather irrelivent higher taxa.
However, if no interactions are found at the species level it might be worth removing this restriction.

In [86]:
def fetchInteractionForTaxon(taxon, interactDataTaxon):
    
    curGlobi = connGlobi.cursor()
    if interactionsToExclude == "":
        curGlobi.execute("SELECT * from globi  WHERE sourceTaxonKingdomName = 'Orthornavirae'  and targetTaxonOrderName = ?;", \
                       (taxon,))  
    else:
        curGlobi.execute("SELECT * from globi  WHERE  sourceTaxonKingdomName = 'Orthornavirae' and targetTaxonOrderName = ? and interactionTypeName NOT IN (?);", \
                     (taxon,interactionsToExclude,))
    interactDataTaxon.extend(curGlobi.fetchall())
    
    curGlobi = connGlobi.cursor()
                         
    if interactionsToExclude == "":
        curGlobi.execute("SELECT * from globi  WHERE targetTaxonKingdomName = 'Orthornavirae' and sourceTaxonOrderName = ?;", \
                     (taxon,))
    else:
        curGlobi.execute("SELECT * from globi  WHERE targetTaxonKingdomName = 'Orthornavirae' and sourceTaxonOrderName = ? and interactionTypeName NOT IN (?);", \
                     (taxon,interactionsToExclude,))
                         
    interactDataTaxon.extend(curGlobi.fetchall())

In [87]:
interactDataTaxon = []

for animal in taxa:
    print(interactDataTaxon)
    fetchInteractionForTaxon(animal,interactDataTaxon)
    

[]


In [88]:
#interactDataTaxon.extend(sources)
len(interactDataTaxon)

14967

In [89]:
# Convert to a Pandas dataframe
interactDataTaxon = pd.DataFrame(interactDataTaxon)

In [90]:
# Add column names
interactDataTaxon
interactDataTaxon.columns = ['sourceTaxonId', \
                                'sourceTaxonIds','sourceTaxonName','sourceTaxonRank','sourceTaxonPathNames', \
                                'sourceTaxonPathIds','sourceTaxonPathRankNames','sourceTaxonSpeciesName','sourceTaxonSpeciesId',\
                                'sourceTaxonGenusName','sourceTaxonGenusId','sourceTaxonFamilyName','sourceTaxonFamilyId',\
                                'sourceTaxonOrderName','sourceTaxonOrderId','sourceTaxonClassName','sourceTaxonClassId',\
                                'sourceTaxonPhylumName','sourceTaxonPhylumId','sourceTaxonKingdomName','sourceTaxonKingdomId',\
                                'sourceId','sourceOccurrenceId','sourceCatalogNumber','sourceBasisOfRecordId',\
                                'sourceBasisOfRecordName','sourceLifeStageId','sourceLifeStageName','sourceBodyPartId',\
                                'sourceBodyPartName','sourcePhysiologicalStateId','sourcePhysiologicalStateName', \
                                'sourceSexId', 'sourceSexName','interactionTypeName',\
                                'interactionTypeId','targetTaxonId','targetTaxonIds','targetTaxonName',\
                                'targetTaxonRank','targetTaxonPathNames','targetTaxonPathIds','targetTaxonPathRankNames',\
                                'targetTaxonSpeciesName','targetTaxonSpeciesId','targetTaxonGenusName','targetTaxonGenusId',\
                                'targetTaxonFamilyName','targetTaxonFamilyId','targetTaxonOrderName','targetTaxonOrderId',\
                                'targetTaxonClassName','targetTaxonClassId','targetTaxonPhylumName','targetTaxonPhylumId',\
                                'targetTaxonKingdomName','targetTaxonKingdomId','targetId','targetOccurrenceId',\
                                'targetCatalogNumber','targetBasisOfRecordId','targetBasisOfRecordName','targetLifeStageId',\
                                'targetLifeStageName','targetBodyPartId','targetBodyPartName','targetPhysiologicalStateId',\
                                'targetPhysiologicalStateName', 'targetSexId', 'targetSexName',\
                                'decimalLatitude','decimalLongitude','localityId',\
                                'localityName','eventDateUnixEpoch','argumentTypeId','referenceCitation',\
                                'referenceDoi','referenceUrl','sourceCitation','sourceNamespace',\
                                'sourceArchiveURI','sourceDOI','sourceLastSeenAtUnixEpoch','combo']

## Get a list of all the primary interacting species

In [91]:
interactingTaxaData = interactDataTaxon.drop_duplicates()

In [92]:
interactingTaxaData.to_csv('interactingTaxaData.csv', index=False, columns=['sourceTaxonName','targetTaxonName','interactionTypeName','sourceTaxonPhylumName','targetTaxonPhylumName'])

In [93]:
primaryInteractingTaxa = interactingTaxaData['sourceTaxonName'].drop_duplicates()
len(primaryInteractingTaxa)

1479

In [94]:
primaryInteractingTaxa = (primaryInteractingTaxa.append(interactingTaxaData['targetTaxonName']).drop_duplicates())
len(primaryInteractingTaxa)

1860

In [95]:
primaryInteractingTaxa

0                Rousettus bat coronavirus HKU9
1             Pipistrellus bat coronavirus HKU5
2               Scotophilus bat coronavirus 512
3              Rhinolophus bat coronavirus HKU2
4                 Miniopterus bat coronavirus 1
5                        Human coronavirus 229E
6                             Rabies lyssavirus
7                          Lagos bat lyssavirus
12                             Irkut lyssavirus
13                    European bat 1 lyssavirus
14                         Duvenhage lyssavirus
15                            Lloviu cuevavirus
17                                Mucambo virus
19                           Chobar Gorge virus
21                             Orthohepevirus D
26                                  Pegivirus B
27                                 Saboya virus
28                              Rio Bravo virus
39                         Phnom Penh bat virus
41             Murray Valley encephalitis virus
42       Montana myotis leukoencephaliti

In [172]:
def gbifkeyFromGBIF(name):
    try:
        match = species.name_backbone(name=name, limit = 1)

        # if there is no match this is returned from .name_backbone {'confidence': 100, 'matchType': 'NONE', 'synonym': False}
        if match['matchType'] == 'NONE':
            return False
        else:
            return match["usageKey"]
    except ValueError as ve:
        print(ve)
        exit(1)

In [173]:
gbifkeyFromGBIF("Bos taurus")

2441022

## A function to query the globi database

In [98]:
    cleanInteractDataTaxon = interactingTaxaData[['sourceTaxonId', 'sourceTaxonName', 'sourceTaxonSpeciesName', 'sourceTaxonRank',
        'sourceTaxonFamilyName', 'interactionTypeName', 'sourceTaxonPhylumName','targetTaxonPhylumName',
        'targetTaxonName','targetTaxonSpeciesName','targetTaxonRank', 'sourceCitation', 'sourceDOI',
        'sourceTaxonKingdomName','sourceTaxonKingdomId', 'targetTaxonKingdomName','targetTaxonKingdomId'                                        
        ]].dropna(subset=['targetTaxonSpeciesName','sourceTaxonSpeciesName'])

In [99]:
allSourceInteractingTaxa = cleanInteractDataTaxon['sourceTaxonSpeciesName'].drop_duplicates()
allTargetInteractingTaxa = cleanInteractDataTaxon['targetTaxonSpeciesName'].drop_duplicates()

In [100]:
allInteractingTaxa = allSourceInteractingTaxa.append(allTargetInteractingTaxa).drop_duplicates()
allInteractingTaxa.count()

1698

In [101]:
allInteractingTaxa.head()

0       Rousettus bat coronavirus HKU9
1    Pipistrellus bat coronavirus HKU5
2      Scotophilus bat coronavirus 512
3     Rhinolophus bat coronavirus HKU2
4        Miniopterus bat coronavirus 1
dtype: object

In [102]:
# How many different sort of interaction do I have left?
# Checking out all the interaction types
cleanInteractDataTaxon.groupby(cleanInteractDataTaxon['interactionTypeName']).size().sort_values(ascending = False)

interactionTypeName
hasHost       8538
pathogenOf     920
dtype: int64

## Drawing a network of the interactions

Now that I have a list of all the species in the country I can use this as my nodes list for the network diagram.

In [103]:
#networkx seems to be a leading network tool in Python
import networkx as nx
import matplotlib.pyplot as plt

try:
    import pygraphviz
    from networkx.drawing.nx_agraph import write_dot
    print("using package pygraphviz")
except ImportError:
    try:
        import pydot
        from networkx.drawing.nx_pydot import write_dot
        print("using package pydot")
    except ImportError:
        print()
        print("Both pygraphviz and pydot were not found ")
        print("see  https://networkx.github.io/documentation/latest/reference/drawing.html")
        print()

using package pydot


In [104]:
# Create graphic object
G = nx.DiGraph()

## A quick look at the interaction data to see if it is what is expected

In [109]:
#cleanInteractDataTaxon.loc[(cleanInteractDataTaxon["sourceTaxonName"] == 'Apis mellifera') & (cleanInteractDataTaxon["targetTaxonName"] == 'Procyon lotor')]
cleanInteractDataTaxon.loc[(cleanInteractDataTaxon["targetTaxonSpeciesName"] == 'Rhinolophus sinicus')]

,sourceTaxonId,sourceTaxonName,sourceTaxonSpeciesName,sourceTaxonRank,sourceTaxonFamilyName,interactionTypeName,sourceTaxonPhylumName,targetTaxonPhylumName,targetTaxonName,targetTaxonSpeciesName,targetTaxonRank,sourceCitation,sourceDOI,sourceTaxonKingdomName,sourceTaxonKingdomId,targetTaxonKingdomName,targetTaxonKingdomId
16,NCBI:693998,Rhinolophus bat coronavirus HKU2,Rhinolophus bat coronavirus HKU2,species,Coronaviridae,pathogenOf,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Shaw, LP, Wang, AD, Dylus, D, et al. The phylo...",,Orthornavirae,NCBI:2732396,Animalia,EOL:1
200,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208
223,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208
224,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208
234,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208
238,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208
241,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208
242,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208
243,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208
247,NCBI:1508227,Bat SARS-like coronavirus,Severe acute respiratory syndrome-related coro...,,Coronaviridae,hasHost,Pisuviricota,Chordata,Rhinolophus sinicus,Rhinolophus sinicus,species,"Chen L, Liu B, Yang J, Jin Q, 2014. DBatVir: t...",,Orthornavirae,NCBI:2732396,Metazoa,NCBI:33208


## Figure out which node has the most records. This is so that the graphic can be scaled.

### Add the nodes to the graph

In [110]:
alltaxa = {}

for index, row in cleanInteractDataTaxon.iterrows():
    alltaxa[row['sourceTaxonName']] = row['sourceTaxonKingdomName']
    
for index, row in cleanInteractDataTaxon.iterrows():
    alltaxa[row['targetTaxonName']] = row['targetTaxonKingdomName']
    

In [177]:
for key in alltaxa:
    if alltaxa[key] == 'Animalia':
        gbifid=gbifkeyFromGBIF(key)
    else:
        gbifid=8
    G.add_node(key, kingdom=alltaxa[key], gbifid=gbifid, label=key if alltaxa[key] == 'Animalia' else "")

### Add edges to the graph

In [178]:
# iterate over the interacting species that are in GBIF and in the country with iterrows()
# Find the taxa found in the country that are in the source taxon name of the interation data,
# then add the edge if the target species is in the country too.

for index, row in cleanInteractDataTaxon.iterrows():
    G.add_edge(row['sourceTaxonName'], row['targetTaxonName'], label = row['interactionTypeName'])
                    
#len(list(G.nodes))


In [179]:
print("Number of nodes = {0}".format(G.number_of_nodes()))
print("Number of edges = {0}".format(G.number_of_edges()))

Number of nodes = 1860
Number of edges = 2428


In [180]:
len(G.nodes)

1860

## Remove any selfloop edges

In [116]:
for e in G.selfloop_edges(data=False):
    G.remove_edge(e[0],e[1])

### For import into Gephi: A nodes and edges file has been replace by the .dot file format

In [181]:
write_dot(G, "..\\docs\\"+"network2"+".dot")

## Make dynamic network that can be explored

In [267]:
filename = "..\\docs\\Chiroptera.html"
with open(filename, "w") as file:
    file.write(" \
<!DOCTYPE html> \
<html> \
<head> \
<script src='cytoscape.min.js'></script> \
<script src='https://unpkg.com/layout-base/layout-base.js'></script> \
<script src='https://unpkg.com/cose-base/cose-base.js'></script> \
<script src='cytoscape-cose-bilkent.js'></script> \
</head> \
<style>#cy {width: 90%; height: 90%; position: absolute; top: 50px; left: 150px;}\
.red {color: red; font-style: bold;} \
.green {color: green; font-style: bold;} \
.blue {color: blue; font-style: bold;} \
.brown {color: brown; font-style: bold;} \
body {font-family: 'times; font-size: 6px;}\
</style> \
<body> \
<h1><em font-style: italic;>Chiroptera and Orthornavirae</h1>")

In [268]:
with open(filename, "a") as file:
    file.write(" \
<div id='cy'></div> \
<script> \
var cy = cytoscape({ \
  container: document.getElementById('cy'), \n \
  elements: [ \
")

In [269]:
file = open(filename, "a")
for n in G.nodes:
        file.write("{ data: { id: '"+n+"', kingdom: '"+str(nx.get_node_attributes(G, 'kingdom')[n])+"', href: 'https://www.gbif.org/species/"+str(nx.get_node_attributes(G, 'gbifid')[n])+"' }, selectable: false,},\n")
file.close()

In [270]:
file = open(filename, "a")
for edge in G.edges:
    file.write("{data: {id: '"+edge[0]+edge[1]+"', source: '"+edge[0]+"', target: '"+edge[1]+"', href: 'https://www.globalbioticinteractions.org/?interactionType="+str(nx.get_edge_attributes(G, 'label')[edge])+"&sourceTaxon="+edge[0]+"&targetTaxon="+edge[1]+"', label: '"+nx.get_edge_attributes(G, 'label')[edge]+"'}},\n")
file.close()

In [271]:
with open(filename, "a") as file:
    file.write("], \
style: [ \n\
        { \n\
            selector: 'node', \n\
            style: { \n\
                shape: 'circle', \n\
                'background-color': 'yellow', \n\
                label: 'data(id)', \n\
                'text-valign': 'center', \n\
                'font-family': 'helvetica', \n\
                'font-style': 'italic', \n\
                'font-size': '6px', \n\
                'border-width': '0.5', \n\
                'border-color': 'black', \n\
                'width': '10', \n\
                'height': '10' \n\
            } \n\
        },  \n\
        {  \n\
            selector: 'edge',  \n\
            style: {  \n\
                'font-family': 'helvetica', \n\
                'font-size': '6px', \n\
                'color': 'blue', \n\
                'curve-style': 'bezier', \n\
                'target-arrow-shape': 'triangle',  \n\
                'width': '1' \n\
                } \n\
         },  \n\
         {  \n\
              selector: '[kingdom = \"Orthornavirae\"]',   \n\
              css: {  \n\
                'background-color': 'lightgreen'  \n\
                 }  \n\
        }, \n\
        {  \n\
              selector: '[kingdom = \"Animalia\"]',   \n\
              css: {  \n\
                'background-color': 'lightblue'  \n\
                 }  \n\
        }, \n\
        {  \n\
              selector: ':selected',   \n\
              css: {  \n\
                'line-color': 'red',  \n\
                label: 'data(id)', \n\
                'shape': 'square',  \n\
                'background-color': 'red'  \n\
                 }  \n\
        }], \n\
layout:  { \n\
            name: 'circle', padding: 1, animate: true, gravity: 30, animationDuration: 3000 \n\
     } \n\
} \n\
); \n\
")

In [272]:
with open(filename, "a") as file:
    file.write("cy.userZoomingEnabled( true ); \n\
    cy.getElementById('Chiroptera').select(); \n\
    cy.on('tap', 'node', function(){ \n\
        try { // your browser may block popups \n\
              window.open( this.data('href') ); \n\
            } catch(e){ // fall back on url change \n\
        window.location.href = this.data('href'); \n\
    } \n\
}); \n\
    cy.on('tap', 'edge', function(){  \n\
        try { // your browser may block popups  \n\
              window.open( this.data('href') );  \n\
            } catch(e){ // fall back on url change  \n\
        window.location.href = this.data('href'); \n\
    } \n\
}); \n\
</script> \n\
")

In [273]:
with open(filename, "a") as file:
    file.write("</ul> \
        <br /> \
        </body> \
        </html>")

In [274]:
file.close()